In [4]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/forest-cover-type-prediction/train.csv
/kaggle/input/forest-cover-type-prediction/sampleSubmission.csv
/kaggle/input/forest-cover-type-prediction/test.csv


[original kernel](https://www.kaggle.com/nadare/eda-feature-engineering-and-modeling-4th-359)

# Summary
Stacking 10 models using LightGBM.

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import gaussian_kde
from tqdm import tqdm_notebook
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn import datasets
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

import warnings
import gc
from multiprocessing import Pool, cpu_count
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import lightgbm as lgb

import lightgbm as lgb

from tqdm import tqdm

import os
import gc
from itertools import combinations, chain
from datetime import datetime

from itertools import chain, combinations

warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
filename_train = '/kaggle/input/forest-cover-type-prediction/train.csv'
filename_test = '/kaggle/input/forest-cover-type-prediction/test.csv'
filename_submission = '/kaggle/input/forest-cover-type-prediction/sampleSubmission.csv'

# nadara's kenel

In [5]:
train_df = pd.read_csv(filename_train)
test_df = pd.read_csv(filename_test)
smpsb = pd.read_csv(filename_submission)

### EDA of Preprocessing

#### EDA and LeaderBoard Hacking

In [ ]:
print(train_df.columns)
print()
print(train_df.columns[1:11])

features_terrain = train_df.columns[1:11]

In [ ]:
# First of all, let's see the distribution of each variable.
# You can see there is a big difference in distribution between
# training data and test data.


def compare_dist(ax, feature, train_df=train_df, test_df=test_df):
    sns.kdeplot(train_df[feature], label='train', ax=ax)
    sns.kdeplot(test_df[feature], label='test', ax=ax)
    

def numeric_tile(plot_func, features):
    base_size = 3
    fig, axes = plt.subplots(3, 4, figsize=(base_size*5, base_size*3))
    axes = axes.flatten()
    
    for i, (ax, col) in enumerate(zip(axes, features)):
        plot_func(ax, col)
        ax.set_title(col)
        
    plt.tight_layout()
    
    
numeric_tile(compare_dist, features_terrain)

In [ ]:
# For the training data, display the distribution of variables for each target.

# Please pay attention to `Elevation`. The difference between the training 
# data and the teset data distribution is thought to be due to the difference between
# the propertion of the target variabels in the training data and the test data.

def compare_target(ax, feature, train_df=train_df, test_df=test_df):
    sns.kdeplot(train_df.loc[:, feature], label='train', ax=ax)
    sns.kdeplot(test_df.loc[:, feature], label='test', ax=ax)
    
    for target in range(1, 7+1):
        indices = (train_df['Cover_Type'] == target)
        sns.kdeplot(train_df.loc[indices, feature], label=target,
                    alpha=0.6, lw=1, ax=ax)
        
        
numeric_tile(compare_target, features_terrain)

# LeaderBoard Probing
Hack the distribution of test dataset by submitting 7 times.

Reference: ["Problems occurring during validation" at 8:30](https://www.coursera.org/learn/competitive-data-science/lecture/8Rp3J/problems-occurring-during-validation)

In [ ]:
# I was able to obtain the distribution of the test data by submitting prediction data with all the same purpose variables.

"""
smpsb = pd.read_csv('/kaggle/input/forest-cover-type-prediction/sampleSubmission.csv')

for i in range(1, 7+1):
    smpsb['Cover_Type'] = i
    smpsb.to_csv('leader_board_probing_class_{}.csv'.format(i), index=None)
"""

# for i in range(1, 7+1):
#     smpsb['Cover_Type'] = i
#     smpsb.to_csv('leader_board_probing_class_{}.csv'.format(i), index=None)


In [ ]:
# The following is the magic number of this competition.
# type_ratio = np.array([0.37053, 0.49681, 0.05936, 0.00103, 0.01295, 0.02687, 0.03242])
type_ratio = np.array([0.37062, 0.49657, 0.05947, 0.00106, 0.01287, 0.02698, 0.03238])

class_weight = {k: v for k, v in enumerate(type_ratio, start=1)}

print('class_weight={}'.format(class_weight))


### Demostrate how to make up a fake test data
Caveat: in the real situation, it only can make up 1 fake feature. 

In [ ]:
# By using these numbers, you can mimic the distribution of the test data from the training data.

def compare_balanced_dist(ax, feature, class_weight=class_weight):
    min_ = min(train_df[feature].min(),
              test_df[feature].min())
    
    max_ = max(train_df[feature].max(),
              test_df[feature].max())
    
    print('Feature {:35s}: min={}, max={}'.format(feature, min_, max_))
    
    X = np.linspace(min_, max_, 1000)
    
    sns.kdeplot(train_df[feature], label='train', ax=ax)
    sns.kdeplot(test_df[feature], label='test', ax=ax)
    btest = np.zeros(1000)
    
    for target in range(1, 7+1):
        indices = (train_df['Cover_Type'] == target)
        
        # Fit train data to a GMM (Gaussian mixture model)
        gaussian_kernel = gaussian_kde(train_df.loc[indices, feature])
        
        # generate PDF using GMM model, then accumlate weighted this PDF.
        btest += gaussian_kernel(X) * class_weight[target]
        
    ax.plot(X, btest, '--', label='balanced', alpha=0.5)
    ax.legend()
    
    
numeric_tile(compare_balanced_dist, features_terrain)

In [ ]:
# By using the following functions, it is possible to perform almost 
# the same evaluation as the leader board even in the local environment.

def balanced_accuracy_score(y_true, y_pred, class_weight=class_weight):
    weights_of_samples = np.apply_along_axis(lambda x: class_weight[x], 
                                                 axis=0, 
                                                 arr=y_true)
    
    return accuracy_score(y_true, y_pred,
                         sample_weight=weights_of_samples)

## Feature Engineering 1

#### Feature: `Aspect`

In [ ]:
sin_ = np.sin(np.pi*train_df['Aspect']/180)
cos_ = np.cos(np.pi*train_df['Aspect']/180)

# However, if this feature quantity alone, the effect seems to be light.

fig, ax = plt.subplots(figsize=(5,4))


for i in range(1, 7+1):
    indices = (train_df['Cover_Type'] == i)
    
    r = 0.5+0.2*i
    
    ax.scatter(cos_[indices]*(r), 
                sin_[indices]*(r), 
                alpha=0.02*r, s=6, label='class {}'.format(i))
    
    
plt.xlim(-2, 3)
ax.legend()
# plt.savefig('aspect.png')

#### Degree to hydrology

In [ ]:
# this may be good feature but unfortunally i forgot to add my data
hydro_h = train_df['Vertical_Distance_To_Hydrology']
hydro_v = train_df['Horizontal_Distance_To_Hydrology']

In [ ]:
plt.scatter(hydro_h, hydro_v, s=1, 
            c=train_df['Cover_Type'], cmap='Set1', alpha=0.3)

In [ ]:
hydro_arctan = np.arctan((hydro_h+0.0001) / (hydro_v+0.0001))

for i in range(1, 7+1):
    indices = (train_df['Cover_Type'] == i)
    sns.kdeplot(hydro_arctan[indices], label='class_{}'.format(i))

In [ ]:
fig, ax = plt.subplots()

ax.scatter(hydro_arctan, np.pi*train_df['Slope']/180,
           c=train_df['Cover_Type'], cmap='Set1', s=1.5, alpha=0.7)


#### Target Encoding

In [ ]:
train_df.filter(regex='Wilder').head()

In [ ]:
plt.figure(figsize=(6,6))

train_df.filter(regex='Wilder').sum(axis=0).plot('pie')

In [ ]:
%%time
# And this is ratio of 'Cover_Type' in each 'Wilderness_area'

# Transform one-hot encodding to label encoding
wilder = (train_df.filter(regex='Wilder') * np.array([1, 2, 3, 4])).sum(axis=1)

print(np.sort(wilder.unique()))
print()
print(wilder.value_counts().sort_index())


### Another way to implement transformation of label encoding

In [ ]:
%%time
wilderness_cols = ['Wilderness_Area1', 'Wilderness_Area2', 
                       'Wilderness_Area3','Wilderness_Area4']

wilderness_dict = {w: index for index, w in enumerate(wilderness_cols, start=1)}

print(wilderness_dict)

train_df[wilderness_cols].idxmax(axis=1).map(wilderness_dict).values

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8,8))
axes = axes.flatten()

for i, ax in enumerate(axes, start=1):
    indices = (wilder == i)
    train_df.loc[indices, 'Cover_Type'].value_counts().sort_index().plot('pie', ax=ax)
    
    ax.set_title('Wilderness Area {}'.format(i))

In [ ]:
# This shows the expression of Soil_Type for the objective variable.

plt.figure(figsize=(12, 4))

sns.heatmap(train_df.iloc[:, -41:].sort_values(by='Cover_Type').iloc[:, :-1].T, cmap='Greys_r')

for i in np.linspace(0, train_df.shape[0], 7+1)[1:]:
    plt.axvline(i, c='r')

As indicated above, categorical values are considered to have a major role in classification.

Therefore, in order to handle categorical values effectively, the **ratio of object variables in each category** is added as a feature quantity.

In order to prevent data leakage and not to excessively trust categorical values which have only a small number, we added values for 10 data as prior distribution to each category.

In [ ]:
def categorical_post_mean(x, type_ratio=type_ratio):
    p = (x.values) * type_ratio
    p = p/p.sum()*x.sum() + 10*type_ratio
    
    return p/p.sum()

# Preprocessing

In [ ]:
train_df = pd.read_csv('/kaggle/input/forest-cover-type-prediction/train.csv')
test_df = pd.read_csv('/kaggle/input/forest-cover-type-prediction/test.csv')
smpsb = pd.read_csv('/kaggle/input/forest-cover-type-prediction/sampleSubmission.csv')

In [ ]:
%%time
from itertools import combinations

def main(train_def, test_df, type_ratio):
    total_df = pd.concat([train_df.drop(columns=['Cover_Type']),
                          test_df])
    
    # Aspect: Axis Tranformation
    total_df['Aspect_Sin'] = np.sin(np.pi*total_df['Aspect']/180)
    total_df['Aspect_Cos'] = np.cos(np.pi*total_df['Aspect']/180)
    
    # Hillshape: statistical feature engineering
    hillshade_col = ['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']
    
    # Pick combination of `hillshade_col` with size of 2.
    for col1, col2 in combinations(hillshade_col, 2):
        total_df[col1 + '_add_' + col2] = total_df[col2] + total_df[col1]
        total_df[col1 + '_dif_' + col2] = total_df[col2] - total_df[col1]
        total_df[col1 + '_div_' + col2] = (total_df[col2]+0.01) / (total_df[col1]+0.01)
        total_df[col1 + '_abs_' + col2] = np.abs(total_df[col2] - total_df[col1])
        
    
    total_df['Hillshade_mean'] = total_df[hillshade_col].mean(axis=1)
    total_df['Hillshade_std'] = total_df[hillshade_col].std(axis=1)
    total_df['Hillshade_max'] = total_df[hillshade_col].max(axis=1)
    total_df['Hillshade_min'] = total_df[hillshade_col].min(axis=1)
    
    
    # Hydrology: Axis Transformation
    total_df['Degree_to_Hydrology'] = ((total_df['Vertical_Distance_To_Hydrology'] + 0.001) /
                                      (total_df['Horizontal_Distance_To_Hydrology']+0.01))
    
    # Horizontal: statistical feature engineering
    horizontal_col = ["Horizontal_Distance_To_Hydrology",
                      "Horizontal_Distance_To_Roadways",
                      "Horizontal_Distance_To_Fire_Points"]
    
    for col1, col2 in combinations(hillshade_col, 2):
        total_df[col1 + "_add_" + col2] = total_df[col2] + total_df[col1]
        total_df[col1 + "_dif_" + col2] = total_df[col2] - total_df[col1]
        total_df[col1 + "_div_" + col2] = (total_df[col2]+0.01) / (total_df[col1]+0.01)
        total_df[col1 + "_abs_" + col2] = np.abs(total_df[col2] - total_df[col1])
        
    
    def categorical_post_mean(x, type_ratio=type_ratio):
        p = (x.values)*type_ratio
        p = p/p.sum()*x.sum() + 10*type_ratio
        return p/p.sum()
    
    
    # Wilderness Area: target encoding
    wilderness_cols = ['Wilderness_Area1', 'Wilderness_Area2', 
                       'Wilderness_Area3','Wilderness_Area4']
    
    # Transform one-hot encoding to label encoding
    wilder = pd.DataFrame([(train_df.iloc[:, 11:15] * np.arange(1, 5)).sum(axis=1),
                          train_df.Cover_Type]).T
    
    wilder.columns = ['Wilder_Type', 'Cover_Type']
    wilder['one'] = 1
    
    piv = wilder.pivot_table(values='one',
                            index='Wilder_Type',
                            columns='Cover_Type',
                            aggfunc='sum').fillna(0)
    
    tmp = pd.DataFrame(piv.apply(categorical_post_mean, axis=1).tolist()).reset_index()
    tmp['index'] = piv.sum(axis=1).index
    tmp.columns = ['Wilder_Type'] + ['Wilder_prob_ctype_{}'.format(i) for i in range(1, 7+1)]
    
    tmp['Wilder_Type_count'] = piv.sum(axis=1).values
    
    total_df['Wilder_Type'] = (total_df[wilderness_cols]*np.arange(1, len(wilderness_cols)+1)).sum(axis=1)
    total_df = total_df.merge(tmp, on='Wilder_Type', how='left')
    
    for i in range(7):
        total_df.loc[:, "Wilder_prob_ctype_{}".format(i+1)] = total_df.loc[:, "Wilder_prob_ctype_{}".format(i+1)].fillna(type_ratio[i])
        
    total_df.loc[:, "Wilder_Type_count"] = total_df.loc[:, "Wilder_Type_count"].fillna(0)
    
      
    # Soil type: target encoding
    soil = pd.DataFrame([(train_df.iloc[:, -41:-1] * np.arange(1, 41)).sum(axis=1),
                          train_df.Cover_Type]).T
    soil.columns = ["Soil_Type", "Cover_Type"]
    soil["one"] = 1
    piv = soil.pivot_table(values="one",
                           index="Soil_Type",
                           columns="Cover_Type",
                           aggfunc="sum").fillna(0)
    
    tmp = pd.DataFrame(piv.apply(categorical_post_mean, axis=1).tolist()).reset_index()
    tmp["index"] = piv.sum(axis=1).index
    tmp.columns = ["Soil_Type"] + ["Soil_prob_ctype_{}".format(i) for i in range(1, 8)]
    tmp["Soil_Type_count"] = piv.sum(axis=1).values
    
    total_df["Soil_Type"] = (total_df.filter(regex="Soil") * np.arange(1, 41)).sum(axis=1)
    total_df = total_df.merge(tmp, on="Soil_Type", how="left")
    
    for i in range(7):
        total_df.loc[:, "Soil_prob_ctype_{}".format(i+1)] = total_df.loc[:, "Soil_prob_ctype_{}".format(i+1)].fillna(type_ratio[i])
    total_df.loc[:, "Soil_Type_count"] = total_df.loc[:, "Soil_Type_count"].fillna(0)
    
    icol = total_df.select_dtypes(np.int64).columns
    fcol = total_df.select_dtypes(np.float64).columns
    total_df.loc[:, icol] = total_df.loc[:, icol].astype(np.int32)
    total_df.loc[:, fcol] = total_df.loc[:, fcol].astype(np.float32)
    
    return total_df
    
    
total_df = main(train_df, test_df, type_ratio=np.array([0.37053, 0.49681, 0.05936, 0.00103, 0.01295, 0.02687, 0.03242]))


In [ ]:
y = train_df["Cover_Type"].values
X = total_df[total_df["Id"] <= 15120].drop("Id", axis=1)
X_test = total_df[total_df["Id"] > 15120].drop("Id", axis=1)

In [ ]:
print(y.shape)
print(X.shape)
print(X_test.shape)

## Layer 1: KNN probability features and Decision tree feature

For the variable created up to the above, the decision tree and the k-nearest neighbor method are applied after narrowing down the number of variables and adding the prediction probability as the feature amount.

I decided the combination of variables to be used last and the setting of parameters based on Multi-class logarithmic loss while considering diversity.


#### KNN_feature

In [ ]:
all_set =  [['Elevation', 500],
            ['Horizontal_Distance_To_Roadways', 500],
            ['Horizontal_Distance_To_Fire_Points', 500],
            ['Horizontal_Distance_To_Hydrology', 500],
            ['Hillshade_9am', 500],
            ['Aspect', 500],
            ['Hillshade_3pm', 500],
            ['Slope', 500],
            ['Hillshade_Noon', 500],
            ['Vertical_Distance_To_Hydrology', 500],
            ['Elevation_PLUS_Vertical_Distance_To_Hydrology', 200],
            ['Elevation_PLUS_Hillshade_9am_add_Hillshade_Noon', 200],
            ['Elevation_PLUS_Aspect', 200],
            ['Elevation_PLUS_Hillshade_Noon_dif_Hillshade_3pm', 200],
            ['Elevation_PLUS_Hillshade_Noon_abs_Hillshade_3pm', 200],
            ['Elevation_PLUS_Hillshade_9am', 200],
            ['Elevation_PLUS_Horizontal_Distance_To_Hydrology', 200],
            ['Elevation_PLUS_Horizontal_Distance_To_Roadways', 100],
            ['Elevation_PLUS_Vertical_Distance_To_Hydrology', 200],
            ['Wilder_Type_PLUS_Elevation', 500],
            ['Wilder_Type_PLUS_Hillshade_Noon_div_Hillshade_3pm', 500],
            ['Wilder_Type_PLUS_Degree_to_Hydrology', 200],
            ['Wilder_Type_PLUS_Hillshade_9am_div_Hillshade_3pm', 500],
            ['Wilder_Type_PLUS_Aspect_Cos', 500],
            ['Hillshade_9am_dif_Hillshade_Noon_PLUS_Hillshade_Noon_dif_Hillshade_3pm', 200],
            ['Hillshade_Noon_PLUS_Hillshade_3pm', 200],
            ['Hillshade_Noon_add_Hillshade_3pm_PLUS_Hillshade_Noon_dif_Hillshade_3pm', 200]
           ]


def simple_feature_scores2(clf, cols, test=False, **params):
    scores = []
    bscores = []
    lscores = []
    
    X_preds = np.zeros((len(y), 7))
    scl = StandardScaler().fit(X.loc[:, cols])
    
    for train, val in StratifiedKFold(n_splits=10, shuffle=True, random_state=2018).split(X, y):
        X_train = scl.transform(X.loc[train, cols])
        X_val = scl.transform(X.loc[val, cols])
        y_train = y[train]
        y_val = y[val]
        C = clf(**params) 

        C.fit(X_train, y_train)
        X_preds[val] = C.predict_proba(X_val)
        #scores.append(accuracy_score(y_val, C.predict(X_val)))
        #bscores.append(balanced_accuracy_score(y_val, C.predict(X_val)))
        #lscores.append(log_loss(y_val, C.predict_proba(X_val), labels=list(range(1, 8))))
    
    if test:
        X_test_select = scl.transform(X_test.loc[:, cols])
        C = clf(**params)
        C.fit(scl.transform(X.loc[:, cols]), y)
        X_test_preds = C.predict_proba(X_test_select)
    else:
        X_test_preds = None
    return scores, bscores, lscores, X_preds, X_test_preds

In [ ]:
%%time 
preds = []
test_preds = []

for colname, neighbor in tqdm_notebook(all_set):
    gc.collect()
    
    params = {'n_neighbors': neighbor,
#              'n_jobs': cpu_count()
             }

    %prun ts, tbs, ls, pred, test_pred = simple_feature_scores2(KNeighborsClassifier, \
                                                          colname.split("_PLUS_"), \
                                                          test=True, \
                                                          **params)
 
    preds.append(pred)
    test_preds.append(test_pred)

In [ ]:
cols = list(chain.from_iterable([[col[0] + "_KNN_{}".format(i) for i in range(1, 8)] for col in all_set]))
knn_train_df = pd.DataFrame(np.hstack(preds)).astype(np.float32)
knn_train_df.columns = cols
knn_test_df = pd.DataFrame(np.hstack(test_preds)).astype(np.float32)
knn_test_df.columns = cols

### Layer 1: Decision Tree Features

In [ ]:
all_set = [['Elevation', 4],
           ['Horizontal_Distance_To_Roadways', 4],
           ['Horizontal_Distance_To_Fire_Points', 3],
           ['Horizontal_Distance_To_Hydrology', 4],
           ['Hillshade_9am', 3],
           ['Vertical_Distance_To_Hydrology', 3],
           ['Slope', 4],
           ['Aspect', 4],
           ['Hillshade_3pm', 3],
           ['Hillshade_Noon', 3],
           ['Degree_to_Hydrology', 3],
           ['Hillshade_Noon_dif_Hillshade_3pm', 3],
           ['Hillshade_Noon_abs_Hillshade_3pm', 3],
           ['Elevation_PLUS_Hillshade_9am_add_Hillshade_Noon', 5],
           ['Elevation_PLUS_Hillshade_max', 5],
           ['Elevation_PLUS_Horizontal_Distance_To_Hydrology', 5],
           ['Aspect_Sin_PLUS_Aspect_Cos_PLUS_Elevation', 5],
           ['Elevation_PLUS_Horizontal_Distance_To_Fire_Points', 5],
           ['Wilder_Type_PLUS_Elevation', 5],
           ['Elevation_PLUS_Hillshade_9am', 5],
           ['Elevation_PLUS_Degree_to_Hydrology', 5],
           ['Wilder_Type_PLUS_Horizontal_Distance_To_Roadways', 5],
           ['Wilder_Type_PLUS_Hillshade_9am_add_Hillshade_Noon', 4],
           ['Wilder_Type_PLUS_Horizontal_Distance_To_Hydrology', 5],
           ['Wilder_Type_PLUS_Hillshade_Noon_abs_Hillshade_3pm', 4],
           ['Hillshade_9am_add_Hillshade_Noon_PLUS_Hillshade_std', 4],
           ['Hillshade_9am_PLUS_Hillshade_9am_add_Hillshade_Noon', 4],
           ['Hillshade_9am_add_Hillshade_Noon_PLUS_Hillshade_Noon_add_Hillshade_3pm', 5]]

def simple_feature_scores(clf, cols, test=False, **params):
    scores = []
    bscores = []
    lscores = []
    
    X_preds = np.zeros((len(y), 7))
    
    
    for train, val in StratifiedKFold(n_splits=10, shuffle=True, random_state=2018).split(X, y):
        X_train = X.loc[train, cols]
        X_val = X.loc[val, cols]
        y_train = y[train]
        y_val = y[val]
        C = clf(**params) 

        C.fit(X_train, y_train)
        X_preds[val] = C.predict_proba(X_val)
        #scores.append(accuracy_score(y_val, C.predict(X_val)))
        #bscores.append(balanced_accuracy_score(y_val, C.predict(X_val)))
        #lscores.append(log_loss(y_val, C.predict_proba(X_val), labels=list(range(1, 8))))
    
    if test:
        X_test_select = X_test.loc[:, cols]
        C = clf(**params)
        C.fit(X.loc[:, cols], y)
        X_test_preds = C.predict_proba(X_test_select)
    else:
        X_test_preds = None
    return scores, bscores, lscores, X_preds, X_test_preds

In [ ]:
preds = []
test_preds = []
for colname, depth in tqdm(all_set):
    #print(colname, depth)
    ts, tbs, ls, pred, test_pred = simple_feature_scores(DecisionTreeClassifier,
                                                         colname.split("_PLUS_"),
                                                         test=True,
                                                         max_depth=depth)
    preds.append(pred)
    test_preds.append(test_pred)

cols = list(chain.from_iterable([[col[0] + "_DT_{}".format(i) for i in range(1, 8)] for col in all_set]))
dt_train_df = pd.DataFrame(np.hstack(preds)).astype(np.float32)
dt_train_df.columns = cols

dt_test_df = pd.DataFrame(np.hstack(test_preds)).astype(np.float32)
dt_test_df.columns = cols

In [ ]:
# target encoding features(1.2.3)
te_train_df = total_df.filter(regex="ctype").iloc[:len(train_df)]
te_test_df = total_df.filter(regex="ctype").iloc[len(train_df):]

In [ ]:
train_level2 = train_df[["Id"]]
test_level2 = test_df[["Id"]]

### Modeling

6 models

- without KNN&DT features
    1. Random Forest Classifier
    1. PCA & K-nearest Neighbors Classifier
    1. LightGBM


- with KNN & DT features
    1. Random Forest Classifier
    1. Logistic Regression
    1. LightGBM

Using these learning machines, data for stacking was created using 10-fold cross validation.

#### without KNN&DT feature


In [ ]:
y = train_df["Cover_Type"].values
X = total_df[total_df["Id"] <= 15120].drop("Id", axis=1)
X_test = total_df[total_df["Id"] > 15120].drop("Id", axis=1)
type_ratio = np.array([0.37053, 0.49681, 0.05936, 0.00103, 0.01295, 0.02687, 0.03242])
class_weight = {k: v for k, v in enumerate(type_ratio, start=1)}

Layer 2: Random forest classifier

In [ ]:
RFC1_col = ["RFC1_{}_proba".format(i) for i in range(1, 8)]
for col in RFC1_col:
    train_level2.loc[:, col] = 0
    test_level2.loc[:, col] = 0

In [ ]:
rfc = RandomForestClassifier(n_estimators=150,
                             max_depth=12,
                             class_weight=class_weight,
                             n_jobs=-1)

confusion = np.zeros((7, 7))
scores = []
for train, val in tqdm(StratifiedKFold(n_splits=10, random_state=2434, shuffle=True).split(X, y)):
    X_train = X.iloc[train, :]
    X_val = X.iloc[val, :]

    y_train = y[train]
    y_val = y[val]
    rfc.fit(X_train, y_train)
    y_val_pred = rfc.predict(X_val)
    y_val_proba = rfc.predict_proba(X_val)
    
    confusion += confusion_matrix(y_val, y_val_pred)    
    train_level2.loc[val, RFC1_col] = y_val_proba
    scores.append(balanced_accuracy_score(y_val, y_val_pred))

rfc.fit(X, y)
test_level2.loc[:, RFC1_col] = rfc.predict_proba(X_test)
#smpsb.loc[:, "Cover_Type"] = rfc.predict(X_test)
#smpsb.to_csv("RFC1.csv", index=None)

In [ ]:
print(np.mean(scores))
sns.heatmap(confusion, cmap="Blues")

### Lyaer 2: PCA & KNN

In [ ]:
KNN1_col = ["KNN1_{}_proba".format(i) for i in range(1, 8)]
for col in KNN1_col:
    train_level2.loc[:, col] = 0
    test_level2.loc[:, col] = 0

In [ ]:
cat_col = X.filter(regex="Soil_Type|Wilderness").columns.tolist()[:-1] + ["Wilder_Type"]

In [ ]:
knn = KNeighborsClassifier(n_neighbors=2, n_jobs=-1)

scl = StandardScaler().fit(X_test.drop(cat_col, axis=1))
X_scl = scl.transform(X.drop(cat_col, axis=1))
X_test_scl = scl.transform(X_test.drop(cat_col, axis=1))
pca = PCA(n_components=23).fit(X_test_scl)
X_pca = pca.transform(X_scl)
X_test_pca = pca.transform(X_test_scl)

confusion = np.zeros((7, 7))
scores = []
for train, val in tqdm(StratifiedKFold(n_splits=10, random_state=2434, shuffle=True).split(X, y)):
    X_train = X_pca[train]
    X_val = X_pca[val]

    y_train = y[train]
    y_val = y[val]
    knn.fit(X_train, y_train)
    y_val_pred = knn.predict(X_val)
    y_val_proba = knn.predict_proba(X_val)
    
    confusion += confusion_matrix(y_val, y_val_pred)    
    train_level2.loc[val, KNN1_col] = y_val_proba
    scores.append(balanced_accuracy_score(y_val, y_val_pred))

knn.fit(X_pca, y)
test_level2.loc[:, KNN1_col] = knn.predict_proba(X_test_pca)
#smpsb.loc[:, "Cover_Type"] = knn.predict(X_test_pca)
#smpsb.to_csv("KNN1.csv", index=None)

In [ ]:
print(np.mean(scores))
sns.heatmap(confusion, cmap="Blues")

### Layer 2: LightGBM

In [ ]:
LGBM1_col = ["LGBM1_{}_proba".format(i) for i in range(1, 8)]
for col in LGBM1_col:
    train_level2.loc[:, col] = 0
    test_level2.loc[:, col] = 0

In [ ]:
cat_col = X.filter(regex="Soil_Type|Wilderness").columns.tolist()[:-1] + ["Wilder_Type"]
categorical_feature = [29, 38]
lgbm_col = X.drop(cat_col[:-2], axis=1).columns.tolist()
class_weight_lgbm = {i: v for i, v in enumerate(type_ratio)}

In [ ]:
gbm = lgb.LGBMClassifier(n_estimators=15,
                         num_class=7,
                         learning_rate=0.1,
                         bagging_fraction=0.6,
                         num_boost_round=370,
                         max_depth=8,
                         max_cat_to_onehot=40,
                         class_weight=class_weight_lgbm,
                         device="cpu",
                         n_jobs=4,
                         silent=-1,
                         verbose=-1)

confusion = np.zeros((7, 7))
scores = []
for train, val in tqdm(StratifiedKFold(n_splits=10, random_state=2434, shuffle=True).split(X, y)):
    X_train = X.loc[train, lgbm_col]
    X_val = X.loc[val, lgbm_col]

    y_train = y[train]
    y_val = y[val]
    gbm.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)],
            verbose=50, categorical_feature=categorical_feature)

    y_val_pred = gbm.predict(X_val)
    y_val_proba = gbm.predict_proba(X_val)
    
    scores.append(balanced_accuracy_score(y_val, y_val_pred))
    confusion += confusion_matrix(y_val, y_val_pred)
    train_level2.loc[val, LGBM1_col] = y_val_proba


X_all = X.loc[:, lgbm_col]
X_test_lgbm = X_test.loc[:, lgbm_col]
gbm.fit(X_all, y, verbose=50, categorical_feature=categorical_feature)
test_level2.loc[:, LGBM1_col] = gbm.predict_proba(X_test_lgbm)
#smpsb["Cover_Type"] = gbm.predict(X_test_lgbm)
#smpsb.to_csv("LGBM1.csv")

In [ ]:
print(np.mean(scores))
sns.heatmap(confusion, cmap="Blues")

### with KNN & DT features


In [ ]:
X_p = pd.concat([knn_train_df, dt_train_df, te_train_df], axis=1).astype(np.float32)
X_test_p = pd.concat([knn_test_df, dt_test_df, te_test_df.reset_index(drop=True)], axis=1).astype(np.float32)

#### Layer 2: RandomForestClassifier

In [ ]:
KNNDT_RF_col = ["KNNDT_RF_{}_proba".format(i) for i in range(1, 8)]
for col in KNNDT_RF_col:
    train_level2.loc[:, col] = 0
    test_level2.loc[:, col] = 0

In [ ]:
rfc = RandomForestClassifier(n_jobs=-1,
                             n_estimators=200,
                             max_depth=None,
                             max_features=.7,
                             max_leaf_nodes=220,
                             class_weight=class_weight)

confusion = np.zeros((7, 7))
scores = []
for train, val in tqdm(StratifiedKFold(n_splits=10, shuffle=True, random_state=2434).split(X_p, y)):
    X_train = X_p.iloc[train, :]
    y_train = y[train]
    X_val = X_p.iloc[val, :]
    y_val = y[val]
    rfc.fit(X_train, y_train)

    y_pred = rfc.predict(X_val)
    scores.append(balanced_accuracy_score(y_val, y_pred))
    confusion += confusion_matrix(y_val, y_pred)
    train_level2.loc[val, KNNDT_RF_col] = rfc.predict_proba(X_val)

rfc.fit(X_p, y)
test_level2.loc[:, KNNDT_RF_col] = rfc.predict_proba(X_test_p)

In [ ]:
print(np.mean(scores))
sns.heatmap(confusion, cmap="Blues")

#### Layer 2: Logistic Regression

In [ ]:
KNNDT_LR_col = ["KNNDT_LR_{}_proba".format(i) for i in range(1, 8)]
for col in KNNDT_LR_col:
    train_level2.loc[:, col] = 0
    test_level2.loc[:, col] = 0

In [ ]:
confusion = np.zeros((7, 7))
scores = []
for train, val in tqdm(StratifiedKFold(n_splits=10, shuffle=True, random_state=2434).split(X, y)):
    X_train = X_p.iloc[train, :]
    y_train = y[train]
    X_val = X_p.iloc[val, :]
    y_val = y[val]
    lr = LogisticRegression(n_jobs=-1, multi_class="multinomial", C=10**9, solver="saga", class_weight=class_weight)
    lr.fit(X_train, y_train)
    y_val_pred = lr.predict(X_val)
    train_level2.loc[val, KNNDT_LR_col] = lr.predict_proba(X_val)
    scores.append(balanced_accuracy_score(y_val, y_val_pred))
    confusion += confusion_matrix(y_val, y_val_pred)

lr.fit(X_p, y)
test_level2.loc[:, KNNDT_LR_col] = lr.predict_proba(X_test_p)

In [ ]:
print(np.mean(scores))
sns.heatmap(confusion, cmap="Blues")

#### Layer 2: LightGBM

In [ ]:
KNNDT_LGB_col = ["KNNDT_LGB_{}_proba".format(i) for i in range(1, 8)]
for col in KNNDT_LGB_col:
    train_level2.loc[:, col] = 0
    test_level2.loc[:, col] = 0

In [ ]:
X = total_df[total_df["Id"] <= 15120].drop("Id", axis=1)
X_test = total_df[total_df["Id"] > 15120].drop("Id", axis=1).reset_index(drop=True)

X_d = pd.concat([X.drop(total_df.filter(regex="Type\d+").columns, axis=1),
                 knn_train_df,
                 dt_train_df], axis=1)

X_test_d = pd.concat([X_test.drop(total_df.filter(regex="Type\d+").columns, axis=1),
                 knn_test_df,
                 dt_test_df], axis=1)

fcol = X_d.select_dtypes(np.float64).columns
X_d.loc[:, fcol] = X_d.loc[:, fcol].astype(np.float32)
X_d = X_d.values.astype(np.float32)
X_test_d.loc[:, fcol] = X_test_d.loc[:, fcol].astype(np.float32)
X_test_d = X_test_d.values.astype(np.float32)

In [ ]:
class_weight_lgbm = {i: v for i, v in enumerate(type_ratio)}

gbm = lgb.LGBMClassifier(n_estimators=300,
                         num_class=8,
                         num_leaves=32,
                         feature_fraction=0.3,
                         min_child_samples=20,
                         learning_rate=0.05,
                         num_boost_round=430,
                         max_depth=-1,                         
                         class_weight=class_weight_lgbm,
                         device="cpu",
                         n_jobs=4,
                         silent=-1,
                         verbose=-1)

confusion = np.zeros((7, 7))
scores = []
for train, val in tqdm(StratifiedKFold(n_splits=10, shuffle=True, random_state=2434).split(X_p, y)):
    X_train = X_d[train]
    X_val = X_d[val]

    y_train = y[train]
    y_val = y[val]
    gbm.fit(X_train, y_train, categorical_feature=[33, 42])

    y_pred = gbm.predict(X_val)
    scores.append(balanced_accuracy_score(y_val, y_pred))
    confusion += confusion_matrix(y_val, y_pred)
    train_level2.loc[val, KNNDT_LGB_col] = gbm.predict_proba(X_val)
    
gbm.fit(X_d, y, categorical_feature=[33, 42])
test_level2.loc[:, KNNDT_LGB_col] = gbm.predict_proba(X_test_d)

In [ ]:
print(np.mean(scores))
sns.heatmap(confusion, cmap="Blues")

# ykskks's kernel

In [6]:
train=pd.read_csv(filename_train)
test=pd.read_csv(filename_test)

In [ ]:
#drop columns that have the same value in every row
train.drop(['Soil_Type7', 'Soil_Type15'], axis=1, inplace=True)
test.drop(['Soil_Type7', 'Soil_Type15'], axis=1, inplace=True)

In [ ]:
train['HF1'] = train['Horizontal_Distance_To_Hydrology']+train['Horizontal_Distance_To_Fire_Points']
train['HF2'] = abs(train['Horizontal_Distance_To_Hydrology']-train['Horizontal_Distance_To_Fire_Points'])
train['HR1'] = abs(train['Horizontal_Distance_To_Hydrology']+train['Horizontal_Distance_To_Roadways'])
train['HR2'] = abs(train['Horizontal_Distance_To_Hydrology']-train['Horizontal_Distance_To_Roadways'])
train['FR1'] = abs(train['Horizontal_Distance_To_Fire_Points']+train['Horizontal_Distance_To_Roadways'])
train['FR2'] = abs(train['Horizontal_Distance_To_Fire_Points']-train['Horizontal_Distance_To_Roadways'])
train['ele_vert'] = train.Elevation-train.Vertical_Distance_To_Hydrology
train['Mean_Amenities']=(train.Horizontal_Distance_To_Fire_Points + train.Horizontal_Distance_To_Hydrology + train.Horizontal_Distance_To_Roadways) / 3  
train['Mean_Fire_Hyd']=(train.Horizontal_Distance_To_Fire_Points + train.Horizontal_Distance_To_Hydrology) / 2 

In [ ]:
test['HF1'] = test['Horizontal_Distance_To_Hydrology']+test['Horizontal_Distance_To_Fire_Points']
test['HF2'] = abs(test['Horizontal_Distance_To_Hydrology']-test['Horizontal_Distance_To_Fire_Points'])
test['HR1'] = abs(test['Horizontal_Distance_To_Hydrology']+test['Horizontal_Distance_To_Roadways'])
test['HR2'] = abs(test['Horizontal_Distance_To_Hydrology']-test['Horizontal_Distance_To_Roadways'])
test['FR1'] = abs(test['Horizontal_Distance_To_Fire_Points']+test['Horizontal_Distance_To_Roadways'])
test['FR2'] = abs(test['Horizontal_Distance_To_Fire_Points']-test['Horizontal_Distance_To_Roadways'])
test['ele_vert'] = test.Elevation-test.Vertical_Distance_To_Hydrology 
test['Mean_Amenities']=(test.Horizontal_Distance_To_Fire_Points + test.Horizontal_Distance_To_Hydrology + test.Horizontal_Distance_To_Roadways) / 3  
test['Mean_Fire_Hyd']=(test.Horizontal_Distance_To_Fire_Points + test.Horizontal_Distance_To_Hydrology) / 2

In [ ]:
#Id for later use
Id_train=train['Id']
Id_test=test['Id']

train.drop('Id', axis=1, inplace=True)
test.drop('Id', axis=1, inplace=True)

In [ ]:
x_train=train.drop('Cover_Type', axis=1)
y_train=train['Cover_Type']

### randomforest

In [ ]:
#prepare df to store pred proba
x_train_L2=pd.DataFrame(Id_train)
x_test_L2=pd.DataFrame(Id_test)
rf_cul=['rf'+str(i+1) for i in range(7)]

#prepare cols to store pred proba
for i in rf_cul:
    x_train_L2.loc[:, i]=0
    x_test_L2.loc[:, i]=0

rf=RandomForestClassifier(max_depth=None, max_features=20,n_estimators=500, random_state=1)

#StratifiedKfold to avoid leakage
for train_index, val_index in tqdm(StratifiedKFold(n_splits=10, shuffle=True, random_state=1).split(x_train, y_train)):
    x_train_L1=x_train.iloc[train_index, :]
    y_train_L1=y_train.iloc[train_index]
    x_val_L1=x_train.iloc[val_index, :]
    y_val_L1=y_train.iloc[val_index]

    rf.fit(x_train_L1, y_train_L1)
    y_val_proba=rf.predict_proba(x_val_L1)
    x_train_L2.loc[val_index, rf_cul]=y_val_proba

rf.fit(x_train, y_train)
x_test_L2.loc[:, rf_cul]=rf.predict_proba(test)

#### LightGBM

In [ ]:
#prepare df to store pred proba
#x_train_L2=pd.DataFrame(Id_train)
#x_test_L2=pd.DataFrame(Id_test)
lgbm_cul=['lgbm'+str(i+1) for i in range(7)]

#prepare cols to store pred proba
for i in lgbm_cul:
    x_train_L2.loc[:, i]=0
    x_test_L2.loc[:, i]=0

lgbm=lgb.LGBMClassifier(learning_rate=0.3, max_depth=-1, min_child_samples=20, n_estimators=300, num_leaves=200, random_state=1, n_jobs=4)

#StratifiedKfold to avoid leakage
for train_index, val_index in tqdm(StratifiedKFold(n_splits=10, shuffle=True, random_state=1).split(x_train, y_train)):
    x_train_L1=x_train.iloc[train_index, :]
    y_train_L1=y_train.iloc[train_index]
    x_val_L1=x_train.iloc[val_index, :]
    y_val_L1=y_train.iloc[val_index]

    lgbm.fit(x_train_L1, y_train_L1)
    y_val_proba=lgbm.predict_proba(x_val_L1)
    x_train_L2.loc[val_index, lgbm_cul]=y_val_proba

lgbm.fit(x_train, y_train)
x_test_L2.loc[:, lgbm_cul]=lgbm.predict_proba(test)

### Logistic Regression

In [ ]:
lr_cul=['lr'+str(i+1) for i in range(7)]

#prepare cols to store pred proba
for i in lr_cul:
    x_train_L2.loc[:, i]=0
    x_test_L2.loc[:, i]=0
    
pca=PCA(n_components=40)
x_train_pca=pd.DataFrame(pca.fit_transform(x_train))
test_pca=pd.DataFrame(pca.transform(test))

pipeline=Pipeline([('scaler', StandardScaler()), ('lr', LogisticRegression(C=10, solver='newton-cg', multi_class='multinomial',max_iter=500))])

#StratifiedKfold to avoid leakage
for train_index, val_index in tqdm(StratifiedKFold(n_splits=10, shuffle=True, random_state=1).split(x_train_pca, y_train)):
    x_train_L1=x_train_pca.iloc[train_index, :]
    y_train_L1=y_train.iloc[train_index]
    x_val_L1=x_train_pca.iloc[val_index, :]
    y_val_L1=y_train.iloc[val_index]

    pipeline.fit(x_train_L1, y_train_L1)
    y_val_proba=pipeline.predict_proba(x_val_L1)
    x_train_L2.loc[val_index, lr_cul]=y_val_proba

pipeline.fit(x_train_pca, y_train)
x_test_L2.loc[:, lr_cul]=pipeline.predict_proba(test_pca)

### SVM

In [ ]:
svm_cul=['svm'+str(i+1) for i in range(7)]

#prepare cols to store pred proba
for i in svm_cul:
    x_train_L2.loc[:, i]=0
    x_test_L2.loc[:, i]=0
    
#pca=PCA(n_components=40)
#x_train_pca=pca.fit_transform(x_train)
#test_pca=pca.transform(test)

pipeline=Pipeline([('scaler', StandardScaler()), ('svm', SVC(C=10, gamma=0.1, probability=True))])


#StratifiedKfold to avoid leakage
for train_index, val_index in tqdm(StratifiedKFold(n_splits=10, shuffle=True, random_state=1).split(x_train_pca, y_train)):
    x_train_L1=x_train_pca.iloc[train_index, :]
    y_train_L1=y_train.iloc[train_index]
    x_val_L1=x_train_pca.iloc[val_index, :]
    y_val_L1=y_train.iloc[val_index]

    pipeline.fit(x_train_L1, y_train_L1)
    y_val_proba=pipeline.predict_proba(x_val_L1)
    x_train_L2.loc[val_index, svm_cul]=y_val_proba

pipeline.fit(x_train_pca, y_train)
x_test_L2.loc[:, svm_cul]=pipeline.predict_proba(test_pca)

### stacking

#### Level1 summary

In [ ]:
# concatenate two data
train_L2 = pd.concat([x_train_L2.iloc[:, 1:].reset_index(drop=True), train_level2.iloc[:, 1:].reset_index(drop=True)], axis=1)
test_L2 = pd.concat([x_test_L2.iloc[:, 1:].reset_index(drop=True), test_level2.iloc[:, 1:].reset_index(drop=True)], axis=1)
train_L2.to_csv("Wtrain_L2.csv", index=False)
test_L2.to_csv("Wtest_L2.csv", index=False)



In [ ]:
# each models score

y = pd.read_csv("../input/train.csv")["Cover_Type"].values
model_scores = {}
text = []

for i in range(10):
    y_pred = np.argmax(train_L2.iloc[:, 7*i:7*(i+1)].values, axis=1) + 1
    score = balanced_accuracy_score(y, y_pred)
    model_scores[cols[i*7]] = score
    text.append("{}\t{:<.5}".format(train_L2.columns[i*7], score))

print(*text[::-1], sep="\n")
pd.Series(model_scores).plot(kind="barh")
plt.savefig("model_summary.png")

In [ ]:
# bagging with k-fold
scores = []
gbm = lgb.LGBMClassifier(n_estimators=300,
                         num_class=8,
                         num_leaves=25,
                         learning_rate=5,
                         min_child_samples=20,
                         bagging_fraction=.3,
                         bagging_freq=1,
                         reg_lambda = 10**4.5,
                         reg_alpha = 1,
                         feature_fraction=.2,
                         num_boost_round=8000,
                         max_depth=-1,
                         class_weight=class_weight_lgbm,
                         device="cpu",
                         n_jobs=-1,
                         silent=-1,
                         verbose=-1)

proba = np.zeros((wtest.shape[0], 7))
for train, val in tqdm(StratifiedKFold(n_splits=5, shuffle=True, random_state=2434).split(wtrain, y)):
    X_train = wtrain[train]
    X_val = wtrain[val]
    y_train = y[train]
    y_val = y[val]
    gbm.fit(X_train, y_train, verbose=-1, 
            eval_set=[(X_train, y_train), (X_val, y_val)], early_stopping_rounds=20)
    proba += gbm.predict_proba(wtest) / 10
    y_pred = gbm.predict(X_val)
    scores.append(balanced_accuracy_score(y_val, y_pred))

print(np.mean(scores))

In [ ]:


smpsb["Cover_Type"] = np.argmax(proba, axis=1) + 1
smpsb.to_csv("final_submission_bagging.csv", index=False)

